In [2]:
import mmcv
import numpy as np

In [8]:
@mmcv.register_handler('txt')
class TxtHandler1(mmcv.BaseFileHandler):

    def load_from_fileobj(self, file):
        return [line.strip('\n') for line in file.readlines()]

    def dump_to_fileobj(self, obj, file):
        for i in range(len(obj)):
            file.write(str(obj[i])+'\n')

    def dump_to_str(self, obj, **kwargs):
        return str(obj)

In [6]:
img_ids = np.load('./appendix/coco/longtail2017/img_ids.npy')
mmcv.dump(img_ids,file='./appendix/coco/longtail2017/copyDecoupling/copy_decoupling_img_ids.txt')

In [2]:
voc_img_ids = np.load("/home/pengpeng/LongTailMLC/appendix/VOCdevkit/longtail2012/single_img_ids.npy")

In [7]:
voc_img_ids =mmcv.load(file="/home/pengpeng/LongTailMLC/appendix/VOCdevkit/longtail2012/copyDecoupling/img_ids.txt")
mmcv.dump(voc_img_ids,file="/home/pengpeng/LongTailMLC/appendix/VOCdevkit/longtail2012/copyDecoupling/img_ids.pkl")

In [9]:
coco_img_ids = mmcv.load("/home/pengpeng/LongTailMLC/appendix/coco/longtail2017/img_id.txt")

In [10]:
print(len(coco_img_ids))

1909


In [15]:
data = mmcv.load("/home/pengpeng/LTMC/appendix/coco/longtail2017/class_split.pkl")

In [16]:
print(data.keys())

dict_keys(['head', 'middle', 'tail'])


In [17]:
print(data)

{'head': {0, 2, 24, 26, 39, 41, 42, 43, 44, 45, 56, 57, 58, 60, 62, 63, 67, 69, 71, 72, 73, 75}, 'middle': {1, 3, 5, 7, 8, 9, 13, 15, 16, 25, 27, 28, 32, 34, 35, 38, 40, 46, 47, 48, 49, 50, 51, 53, 55, 59, 61, 64, 65, 66, 68, 74, 77}, 'tail': {4, 6, 10, 11, 12, 14, 17, 18, 19, 20, 21, 22, 23, 29, 30, 31, 33, 36, 37, 52, 54, 70, 76, 78, 79}}


In [ ]:
group_ids = mmcv.load("/home/pengpeng/LongTailMLC/appendix/coco/IncLearning/group_ids.pkl")
print(group_ids)

In [30]:
data= mmcv.load("/home/pengpeng/LongTailMLC/appendix/coco/longtail2017/metadata.pkl")

In [34]:
print(data.keys())
print(data['class_split'])

dict_keys(['gt_labels', 'class_freq', 'neg_class_freq', 'condition_prob', 'img_ids', 'cls_data_list', 'class_split'])
[[0, 2, 24, 26, 39, 41, 42, 43, 44, 45, 56, 57, 58, 60, 62, 63, 67, 69, 71, 72, 73, 75], [1, 3, 5, 7, 8, 9, 13, 15, 16, 25, 27, 28, 32, 34, 35, 38, 40, 46, 47, 48, 49, 50, 51, 53, 55, 59, 61, 64, 65, 66, 68, 74, 77], [4, 6, 10, 11, 12, 14, 17, 18, 19, 20, 21, 22, 23, 29, 30, 31, 33, 36, 37, 52, 54, 70, 76, 78, 79]]


In [2]:
from dataset import ClassAwareSampler,getCOCODataset,getVocDataset
import torch

In [4]:
trainDataset,testDataset = getVocDataset(useCopyDecoupling=True)

len(train_dataset)):  2346
len(val_dataset)):  4952


In [6]:
# sampler = ClassAwareSampler(data_source=trainDataset,clsDataListFile="/home/pengpeng/LongTailMLC/appendix/VOCdevkit/longtail2012/class_freq.pkl",num_classes=20)
dataloader = torch.utils.data.DataLoader(trainDataset,batch_size=32,shuffle=False,sampler=None)

In [8]:
for i,(img,target,mask) in enumerate(dataloader):
    print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73


In [71]:
trainDataset, testDataset = getVocDataset(useCopyDecoupling=False)

len(train_dataset)):  1142
len(val_dataset)):  4952


In [11]:
import numpy as np
class_aware_weight = np.load("/home/pengpeng/LongTailMLC/appendix/VOCdevkit/longtail2012/class_aware_weights.npy")
print(class_aware_weight.shape)

(1142,)


In [13]:
import mmcv

In [18]:
class_freq = mmcv.load("/home/pengpeng/LongTailMLC/appendix/VOCdevkit/longtail2012/class_freq.pkl")
gt_labels = class_freq['gt_labels']
arr = np.array(gt_labels)
print(arr.shape)

(1142, 20)


In [19]:
mask = np.load("/home/pengpeng/LongTailMLC/appendix/VOCdevkit/longtail2012/copyDecoupling/masks.npy")
print(mask.shape)

(2346, 20)


In [23]:
# 用于ClassAwareSampling 中制作cls_data_list
class_freq = mmcv.load("/home/pengpeng/LongTailMLC/appendix/coco/longtail2017/class_freq.pkl")
gt_labels = class_freq['gt_labels']
arr = np.array(gt_labels)
num_classes = arr.shape[1]
cls_data_list = [list() for i in range(num_classes)]
cur = 0
for i in range(arr.shape[0]):
    for j in range(num_classes):
        if arr[i][j] == 1:
            cls_data_list[j].append(cur)
            cur += 1
print(cur)

8316


In [24]:
mmcv.dump({'cls_data_list':cls_data_list},"/home/pengpeng/LongTailMLC/appendix/coco/longtail2017/copyDecoupling/cls_data_list.pkl")